In [1]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
import os

dataset_path = "./dataset"

for celebrity_folder in os.listdir(dataset_path):
    celeb_path = os.path.join(dataset_path, celebrity_folder)

    if os.path.isdir(celeb_path):
        counter = 1
        celeb_name_clean = celebrity_folder.replace(" ", "_").lower()

        for entry in sorted(os.scandir(celeb_path), key=lambda e: e.name):
            if entry.is_file() and entry.name.lower().endswith(".jpg"):
                new_name = f"{celeb_name_clean}{counter}.jpg"
                new_path = os.path.join(celeb_path, new_name)

                try:
                    os.rename(entry.path, new_path)
                    counter += 1
                except Exception as e:
                    print(f"❌ Fehler bei {entry.name}: {e}")

In [2]:
img = cv2.imread("./test images/chalamettest1.jpg")
img.shape

(275, 183, 3)

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray.shape

In [ ]:
gray


In [ ]:
plt.imshow(gray, cmap = "gray")
plt.show()

In [ ]:
face_cascade = cv2.CascadeClassifier("./haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("./haarcascades/haarcascade_eye.xml")


In [ ]:
(x,y,w,h) = faces[0]
x,y,w,h

In [ ]:
face_img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)



In [ ]:
cv2.destroyAllWindows()
for (x, y, w, h) in faces:
    face_img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    roi_gray = gray[y:y + h, x:x + w]
    roi_color = face_img[y:y + h, x:x + w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

plt.figure()
plt.imshow(face_img, cmap='gray')
plt.show()


In [ ]:
import cv2
import os

def get_cropped_image_if_2_eyes(image_path):
    image_path = os.path.normpath(image_path)

    if not os.path.exists(image_path):
        print(f"Datei nicht gefunden: {image_path}")
        return None

    img = cv2.imread(image_path)
    if img is None:
        print(f"Konnte Bild nicht laden: {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_color)
        if len(eyes) >= 2:
            return roi_color
    return None


In [ ]:
img2 = cv2.imread("./test images/chalamettest1.jpg")
plt.imshow(img2)
plt.show()

In [ ]:
cropped_img2 = get_cropped_image_if_2_eyes("./test images/Tom Holland Talks The Crowded Room ....jpg")
cropped_img2

In [ ]:
path_to_data = "./dataset/"
path_to_cropped = "./dataset/cropped/"

In [ ]:
import os

img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)


In [ ]:
img_dirs


In [ ]:
import shutil

if os.path.exists(path_to_cropped):
    shutil.rmtree(path_to_cropped)
os.mkdir(path_to_cropped)

In [ ]:

for img_dir in img_dirs:
    celebrity_name = os.path.basename(img_dir)
    print("Celebrity:", celebrity_name)

    for entry in os.scandir(img_dir):
        if entry.is_file():
            image_path = os.path.normpath(entry.path) 
            roi_color = get_cropped_image_if_2_eyes(image_path)

            if roi_color is not None:
                cropped_folder = os.path.join(path_to_cropped, celebrity_name)

                if not os.path.exists(cropped_folder):
                    os.makedirs(cropped_folder)
                    print("Generating:", cropped_folder)

                cropped_file_name = os.path.join(
                    cropped_folder, os.path.basename(image_path)
                )
                cv2.imwrite(cropped_file_name, roi_color)
                

In [ ]:

path_to_cropped = "./dataset/cropped"
celebrity_file_names_dict = {}

for celebrity_folder in os.listdir(path_to_cropped):
    celeb_path = os.path.join(path_to_cropped, celebrity_folder)

    if os.path.isdir(celeb_path):
        celeb_name = celebrity_folder.lower().replace(" ", "_")
        image_paths = []

        for file in sorted(os.listdir(celeb_path)):
            image_path = os.path.join(celeb_path, file)
            image_paths.append(image_path)

        celebrity_file_names_dict[celeb_name] = image_paths

print(celebrity_file_names_dict)


In [ ]:
cr_img = cv2.imread("./dataset/cropped/timothee chalamet/timothee_chalamet6.jpg")
plt.imshow(cr_img)
plt.show()

In [ ]:
import numpy as np
import pywt
import cv2

def w2d(img, mode = "haar", level =1):
    imArray = img

    imArray = cv2.cvtColor(imArray, cv2.COLOR_RGB2GRAY )
    imArray = np.float32(imArray)
    imArray /= 255
    
    coeffs = pywt.wavedec2(imArray, mode, level = level) 

    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0

    imArray_H = pywt.waverec2(coeffs_H, mode)
    imArray_H *= 255;
    imArray_H = np.uint8(imArray_H)

    return imArray_H

In [ ]:
im_har = w2d(cr_img, "db1", 5)
plt.imshow(im_har, cmap= "gray")
plt.show()

In [ ]:
celebrity_file_names_dict
